In [1]:
# imports
import sys
import openmm
import openmm.app as app
import openmm.unit as unit

import numpy as np
import mdtraj

import rdkit.Chem as Chem
import rdkit.Chem.AllChem as AllChem
from rdkit.Geometry import Point3D

In [2]:
# load files
psf = app.CharmmPsfFile('init.psf')
pdbfile = app.PDBFile('init.pdb')
params = app.CharmmParameterSet('toppar/top_all36_prot.rtf', 'toppar/par_all36m_prot.prm', 'toppar/toppar_water_ions.str')
positions = pdbfile.getPositions(asNumpy=True)

In [3]:
# system
psf.setBox(39.0*unit.angstroms, 39.0*unit.angstroms, 39.0*unit.angstroms)

system = psf.createSystem(params,
    nonbondedMethod=app.PME, 
    nonbondedCutoff=1.2*unit.nanometers,
    implicitSolvent=None,
    constraints=app.HBonds,
    rigidWater=True, 
    verbose=True, 
    ewaldErrorTolerance=0.0005)

barostat = openmm.MonteCarloBarostat(1.0*unit.bar, 300.0*unit.kelvin)
system.addForce(barostat)

Adding particles...
Adding constraints...
Adding lonepairs...
Adding bonds...
Adding angles...
    Number of bond constraints: 3685
    Number of angle constraints: 1812
Adding Urey-Bradley terms
Adding torsions...
Adding impropers...
Adding CMAP coupled torsions...
Adding nonbonded interactions...
Build exclusion list...
    Number of 1-2 pairs: 3765
    Number of 1-3 pairs: 2061
    Number of 1-4 pairs: 353


8

In [4]:
# integrator
integrator = openmm.LangevinIntegrator(300.0*unit.kelvin,   # Temperature of head bath
                                       1.0/unit.picosecond, # Friction coefficient
                                       0.002*unit.picoseconds) # Time step

In [5]:
platform = openmm.Platform.getPlatformByName('CUDA')
simulation = app.Simulation(psf.topology, system, integrator, platform)
simulation.context.setPositions(positions)

In [6]:
def print_energy_and_optimize():
    # initial system energy
    print("\ninitial system energy")
    print(simulation.context.getState(getEnergy=True).getPotentialEnergy().in_units_of(unit.kilocalories_per_mole))
    simulation.minimizeEnergy(maxIterations=1000)
    print("\nafter minimization")
    print(simulation.context.getState(getEnergy=True).getPotentialEnergy().in_units_of(unit.kilocalories_per_mole))

In [8]:
print_energy_and_optimize()


initial system energy
-22231.827772624918 kcal/mol

after minimization
-22231.932337634476 kcal/mol


In [9]:
def np_to_mm(arr: np.ndarray, unit: openmm.unit=unit.angstrom):
    wrapped_val = openmm.unit.quantity.Quantity(arr, unit)
    return wrapped_val

In [10]:
nmr_pdbfile = app.PDBFile('1uao_nmr.pdb')
nmr_positions = nmr_pdbfile.getPositions(asNumpy=True)

In [11]:
positions_nm = simulation.context.getState(getPositions=True).getPositions()

In [12]:
positions_nm[:5]

Quantity(value=[Vec3(x=1.236855149269104, y=1.838982105255127, z=1.6340179443359375), Vec3(x=1.313283920288086, y=1.807863473892212, z=1.5707173347473145), Vec3(x=1.1526904106140137, y=1.8491748571395874, z=1.5737768411636353), Vec3(x=1.2183609008789062, y=1.765228033065796, z=1.7049777507781982), Vec3(x=1.2664939165115356, y=1.9700137376785278, z=1.7004144191741943)], unit=nanometer)

In [13]:
delta = (positions_nm[0] - nmr_positions[0])

In [14]:
positions_nm = simulation.context.getState(getPositions=True).getPositions()
positions_nm[:len(nmr_positions)] = nmr_positions + delta
simulation.context.setPositions(positions_nm)
print_energy_and_optimize()


initial system energy
273396033134.01532 kcal/mol

after minimization
-22264.42217989069 kcal/mol


In [10]:
# simulation
reportInterval = 10
nstep = 10
simulation.reporters.append(app.StateDataReporter(sys.stdout, reportInterval, step=True, time=True, potentialEnergy=True, temperature=True, progress=True, remainingTime=True, speed=True, totalSteps=nstep, separator='\t'))
simulation.reporters.append(mdtraj.reporters.HDF5Reporter('run.h5', reportInterval, coordinates=True, time=True, cell=True, potentialEnergy=True, temperature=True))
simulation.reporters.append(mdtraj.reporters.DCDReporter('run.dcd', reportInterval))
simulation.reporters.append(app.pdbreporter.PDBReporter('output.pdb', reportInterval))
simulation.step(nstep)

# save checkpoint
with open('run.chk', 'wb') as f:
    f.write(simulation.context.createCheckpoint())
del simulation # Make sure to close all files

#"Progress (%)"	"Step"	"Time (ps)"	"Potential Energy (kJ/mole)"	"Temperature (K)"	"Speed (ns/day)"	"Time Remaining"
100.0%	10	0.020000000000000004	-92861.81115066266	7.548226400489384	0	--


/home/yppatel/anaconda3/envs/myenv/lib/python3.8/site-packages/mdtraj/utils/validation.py:115: TypeCastPerformanceWarning: Casting xyz dtype=float64 to <class 'numpy.float32'> 
  warnings.warn("Casting %s dtype=%s to %s " % (name, val.dtype, dtype),


In [15]:
pdb = "/home/yppatel/misc/clean_idp_rl/disordered_mol_untrained/mol0.pdb"
mol = Chem.rdmolfiles.MolFromPDBFile(pdb, removeHs=False)
AllChem.EmbedMultipleConfs(mol, numConfs=100, numThreads=-1)

[07:58:43] Molecule does not have explicit Hs. Consider calling AddHs()


In [20]:
for i in range(mol.GetNumConformers()):
    pos_conf = np_to_mm(mol.GetConformer(i).GetPositions()) + delta
    positions_nm = simulation.context.getState(getPositions=True).getPositions()
    positions_nm[:len(pos_conf)] = pos_conf
    simulation.context.setPositions(positions_nm)
    print_energy_and_optimize()


initial system energy
634053845.2963909 kcal/mol

after minimization
-22233.426123485337 kcal/mol

initial system energy
56451314816.6157 kcal/mol

after minimization
-22269.19482568419 kcal/mol

initial system energy
92018877.80117096 kcal/mol

after minimization
-22234.587541984383 kcal/mol

initial system energy
936986158.8719167 kcal/mol

after minimization
-22298.618672481512 kcal/mol

initial system energy
538031940.8986853 kcal/mol

after minimization
-22335.384476735817 kcal/mol

initial system energy
159779122.54304478 kcal/mol

after minimization
-22233.16471096144 kcal/mol

initial system energy
175229151237.26578 kcal/mol

after minimization
-22322.511777166026 kcal/mol

initial system energy
24103967295665.562 kcal/mol

after minimization
-22313.77686440312 kcal/mol

initial system energy
240749508.77631435 kcal/mol

after minimization
-22372.01957472817 kcal/mol

initial system energy
12580952.370960644 kcal/mol

after minimization
-22360.935683714782 kcal/mol

initial s

KeyboardInterrupt: 

In [15]:
system = Chem.rdmolfiles.MolFromPDBFile('init.pdb', removeHs=False)
conf = system.GetConformer(0)
positions_nm = simulation.context.getState(getPositions=True).getPositions()
for i, pos in enumerate(positions_nm):
    conf.SetAtomPosition(i, Point3D(pos.x, pos.y, pos.z))

In [17]:
Chem.rdmolfiles.MolToPDBFile(system, "system.pdb")